
# Batch Lagrangian Relaxation Solver

This notebook solves the Traveling Salesman Problem with Time Windows (TSPTW) for **multiple cases** using Lagrangian relaxation.

## Features
- Tests across 10 cases with randomly generated data for cities, time windows, and travel times.
- Tracks the objective value for each case.
- Outputs the city-to-city routes for clarity.

---


In [ ]:
from gurobipy import Model, GRB, quicksum
from data_generation import generate_data
import numpy as np

# Lagrangian solver for a single case
def solve_tsp(coords, time_windows, travel_time, origin=0, alpha=0.1, max_iterations=20):
    num_cities = len(coords)
    M = sum(np.max(travel_time, axis=1))  # Upper bound for big-M
    N = list(range(num_cities))  # Set of cities

    # Initialize Lagrange multipliers
    lambda_i = np.zeros(num_cities)  # Start with zero multipliers

    # Define model
    model = Model()
    model.setParam("OutputFlag", 0)

    # Decision variables
    x = model.addVars(N, N, vtype=GRB.BINARY, name="x")  # Binary route variables
    z = model.addVars(N, vtype=GRB.CONTINUOUS, name="z")  # Arrival times
    w = model.addVars(N, vtype=GRB.CONTINUOUS, name="w")  # Waiting times
    h = model.addVars(N, vtype=GRB.CONTINUOUS, name="h")  # Late penalties

    # Define Lagrangian objective function
    def lagrangian_objective(lambda_i):
        return (
            quicksum(travel_time[i, j] * x[i, j] for i in N for j in N)
            + quicksum(w[i] + h[i] for i in N)
            + quicksum(
                lambda_i[i] * (quicksum(x[i, j] for j in N if j != i) - 1)
                for i in N
            )
        )

    # Set the objective function
    model.setObjective(lagrangian_objective(lambda_i), GRB.MINIMIZE)

    # Constraints
    for i in N:
        model.addConstr(quicksum(x[i, j] for j in N) == 1, name=f"Outflow_{i}")  # Flow out
        model.addConstr(quicksum(x[j, i] for j in N) == 1, name=f"Inflow_{i}")  # Flow in
        model.addConstr(x[i, i] == 0, name=f"NoSelfLoop_{i}")  # Prevent self-loops

        model.addConstr(w[i] >= time_windows[i, 0] - z[i], name=f"WaitingTime_{i}")  # Waiting time
        model.addConstr(h[i] >= z[i] - time_windows[i, 1], name=f"LatePenalty_{i}")  # Late penalty

    for i in N:
        for j in N:
            if i != j and j != origin:
                model.addConstr(z[j] >= z[i] + travel_time[i, j] * x[i, j] - (M * (1 - x[i, j])), name=f"Subtour_{i}_{j}")

    # Lagrangian relaxation for start and return constraints
    model.addConstr(quicksum(x[origin, j] for j in N if j != origin) == 1, name="StartFromOrigin")
    model.addConstr(quicksum(x[i, origin] for i in N if i != origin) == 1, name="ReturnToOrigin")

    # Subgradient method to update lambda_i
    def update_lambda(lambda_i, x, alpha):
        for i in N:
            constraint_violation = sum(x[i, j].X for j in N if j != i) - 1
            lambda_i[i] += alpha * constraint_violation
        return lambda_i

    # Solve the problem iteratively and track runtime
    solve_time = 0  # Initialize solve time
    for iteration in range(max_iterations):
        model.optimize()

        solve_time += model.Runtime  # Accumulate runtime for each optimization

        if model.status != GRB.OPTIMAL:
            print(f"Iteration {iteration}: Model not optimal")
            break

        lambda_i = update_lambda(lambda_i, x, alpha)

    # Extract the route
    route = np.zeros((num_cities, num_cities))
    for i in N:
        for j in N:
            if x[i, j].X > 0.5:
                route[i, j] = 1

    return model.ObjVal, route, solve_time

from visualizations import plot_tour_graph  # Import the plotting function

# Test with 10 cases
num_cases = 2
num_cities = 11
for test_id in range(num_cases):
    print(f"Test Case {test_id + 1}")
    coords, time_windows, travel_time = generate_data(num_cities, seed=test_id)
    obj_val, route, solve_time = solve_tsp(coords, time_windows, travel_time)
    print(f"Objective Value: {obj_val}")
    print(f"Solve Time: {solve_time:.2f} seconds")
    print("Route:")
    for i, row in enumerate(route):
        for j, val in enumerate(row):
            if val > 0.5:
                print(f"City {i} -> City {j}")
    
    # Add graph visualization
    if route is not None:
        plot_tour_graph(coords, route, title=f"TSPTW Solution for Test Case {test_id + 1}")
    print("\n")


Test Case 1
